In [1]:
from pyspark import SparkContext,SparkConf

stopWords = ['they', 'she', 'he', 'it', 'the', 'as', 'is', 'and']

conf = SparkConf ()
sc = SparkContext.getOrCreate(conf = conf)

21/12/02 04:06:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 04:06:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# read
rdd = sc.wholeTextFiles("./data/*/*")
# filter stop words
rdd = rdd.flatMap(lambda x: [(x[0], word) for word in x[1].lower().split(" ")]).filter(lambda x: x[1] not in stopWords)
# change the order
rdd = rdd.map(lambda x: (x[1], x[0]))

In [3]:
rdd.take(8)

[('project', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('gutenberg', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('ebook', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('of', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('les', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('miserables,', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('by', 'file:/notebooks/data/Hugo/Miserables.txt'),
 ('victor', 'file:/notebooks/data/Hugo/Miserables.txt')]

In [4]:
# add count and reduce
rdd = rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
# change the order
rdd = rdd.map(lambda x : (x[0][0], [(x[0][1], x[1])])).reduceByKey(lambda a, b: a + b)



In [5]:
rdd.take(1)

[('project',
  [('file:/notebooks/data/Hugo/Miserables.txt', 67),
   ('file:/notebooks/data/Hugo/NotreDame_De_Paris.txt', 66),
   ('file:/notebooks/data/shakespeare/comedies/allswellthatendswell', 1),
   ('file:/notebooks/data/shakespeare/histories/2kinghenryiv', 1),
   ('file:/notebooks/data/Tolstoy/anna_karenhina.txt', 65),
   ('file:/notebooks/data/Tolstoy/war_and_peace.txt', 69),
   ('file:/notebooks/data/shakespeare/comedies/muchadoaboutnothing', 1),
   ('file:/notebooks/data/shakespeare/comedies/tempest', 3),
   ('file:/notebooks/data/shakespeare/tragedies/antonyandcleopatra', 1)])]

In [6]:
rdd.saveAsTextFile("output")